In [1]:
import string
import re
import pandas as pd
import os

from utils import basic
from db.get_db_data import GetTableData
from db.config import Config
_Config = Config()

# setup db connection
conn_cur_list = basic.setup_db_connection()
query_path = str(_Config.queries["get_sta_tables"])
query_string = basic.read_query_file(query_path)

# create 
_GetTableData = GetTableData(conn_cur_list[0], conn_cur_list[1])
sta_tables_df = _GetTableData.create_pandas_table(query_string)

print(sta_tables_df.head())

                       table_name
0   sta_euro_dollar_exchange_rate
1                sta_copper_price
2              sta_usa_bond_2year
3  sta_1_year_treasury_rate_yield
4        sta_capacity_utilization


In [2]:
#test_table = sta_tables_df.iloc[1]

In [3]:
query_path = str(_Config.queries["select_table_sample"])
query_string = basic.read_query_file(query_path)
query_string

'select * from __tablename__ order by RANDOM() limit __limit__;'

In [ ]:
def get_sta_dateformats(sta_table):
    query = query_string.replace("__tablename__", sta_table)
    query = query.replace("__limit__", "1000")
    df = _GetTableData.create_pandas_table(query)

    # determine the date formats for all table sample cells
    df['datetype'] = df.iloc[:,0].apply(lambda x: basic.string_to_sql_type(str(x)))
    
    # return the most presented date format
    major_dateformat = df['datetype'].value_counts(ascending=False).to_frame().index.values[0]

    return major_dateformat